In [1]:
using LinearAlgebra
using Distributions
using Optim
using Random
using StatsFuns
using JuMP
using MosekTools
using StatsBase
using SparseArrays # 可选，用于处理稀疏性（如果数据量很大）
using FileIO
using JLD2
using Plots
using LaTeXStrings
using DataFrames, Colors
using StatsPlots   # 提供 boxplot，基于 Plots

include("Data_Generation_PLD.jl")
include("Estimation_PLD.jl")
include("Estimation_PLD_Fast.jl")
include("Models_PLD.jl")
include("Evaluation_PLD.jl")

calculate_profit (generic function with 1 method)

In [2]:
N = 2 # num of product
N_x = 12 # num of product feature
c_l = ones(N_x)  # X * c_l >= d_r
d_r = ones(N) * 5
rev_gap = 0.001
N_u = 5 # num of customer feature
S_train = 200 # num of training samples
S_test = 1 # num of training samples
m = 5 # num of candidates in training samples
N_nonzero = 5 # num of nonzero entries in A
lambda_list = [0.001]
gamma_list = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.8,1.0]
# gamma_list = [0.0]
num_c = 4
instances = 100
Random.seed!(2)
project_dir = "N=$(N)_N_x=$(N_x)_N_u=$(N_u)_S_train=$(S_train)_N_nonzero=$(N_nonzero)/"

"N=2_N_x=12_N_u=5_S_train=200_N_nonzero=5/"

In [3]:
current_dir = pwd()
parent_dir = dirname(current_dir)
grand_pa_dir = dirname(parent_dir)
data_dir = string(dirname(grand_pa_dir), "/Data/Product_Line_Design/")
if !isdir(data_dir)
    mkpath(data_dir)
end
data_dir = string(data_dir,project_dir)
println("Data directory: ", data_dir)

Data directory: /Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Line_Design/N=2_N_x=12_N_u=5_S_train=200_N_nonzero=5/


#### Data Generation

In [4]:
function compute_w(alpha0,alpha,beta,A,z_input)
    nu0 = alpha0 + beta' * z_input;
    nu = alpha .+ A * z_input;
    return nu0,nu
end

compute_w (generic function with 1 method)

In [5]:
Input_Data = Dict()
ins = 1
while ins <= instances
    data_dir_ins = string(data_dir, "instance=$ins/")
    # ****** Data ******
    theta_true, r_params = Generate_Wang_Qi_Max_True_Paras(N_x,N_u,N_nonzero);
    X_train,Y_train,Z_train = Generate_Wang_Qi_Max_True_Data(N_x, N_u, S_train, m,theta_true);
    X_test,Y_test,Z_test = Generate_Wang_Qi_Max_True_Data(N_x, N_u, S_test, m,theta_true);
    z_input = Z_test[1,:];
    nu0_true,nu_true = compute_w(theta_true.alpha0,theta_true.alpha,theta_true.beta,theta_true.A,z_input)
    w_true = [nu0_true;nu_true]

    is_extreme_estimation = false
    Estimate_Dict = Dict()
    for lambda in lambda_list
        # alpha0_hat, alpha_hat, beta_hat, A_hat, opt_result = estimate_parameters(X_train,Y_train,Z_train,lambda, N_x, N_u, initial_theta=randn((N_x+1)*(N_u+1)) * 0.1);
        alpha0_hat, alpha_hat, beta_hat, A_hat, opt_result, X_tilde = estimate_parameters_fast(X_train, Y_train, Z_train, lambda, N_x, N_u);
        nu0_hat,nu_hat = compute_w(alpha0_hat,alpha_hat,beta_hat,A_hat,z_input)
        w_hat = [nu0_hat;nu_hat]

        if norm(w_hat .- w_true,2) >= 10.0
            # println("ins=$(ins)_w_hat=",round.(w_hat,digits=2))
            is_extreme_estimation = true
            break
        else
            # println("ins=$(ins)_w_hat=",round.(w_hat,digits=2))
            Estimate_Dict["alpha0_lambda=$lambda"] = alpha0_hat
            Estimate_Dict["alpha_lambda=$lambda"] = alpha_hat
            Estimate_Dict["beta_lambda=$lambda"] = beta_hat
            Estimate_Dict["A_lambda=$lambda"] = A_hat
            Estimate_Dict["opt_result_lambda=$lambda"] = opt_result
            Estimate_Dict["nu0_lambda=$lambda"] = nu0_hat
            Estimate_Dict["nu_lambda=$lambda"] = nu_hat
        end
    end
    if is_extreme_estimation
        continue
    else
        Input_Data["theta_true_ins=$ins"] = theta_true
        Input_Data["r_params_ins=$ins"] = r_params
        Input_Data["X_train_ins=$ins"] = X_train
        Input_Data["Y_train_ins=$ins"] = Y_train
        Input_Data["Z_train_ins=$ins"] = Z_train
        Input_Data["X_test_ins=$ins"] = X_test
        Input_Data["Y_test_ins=$ins"] = Y_test
        Input_Data["Z_test_ins=$ins"] = Z_test
        Input_Data["Estimate_ins=$ins"] = Estimate_Dict
        Input_Data["nu0_ins=$ins"] = nu0_true
        Input_Data["nu_ins=$ins"] = nu_true
        ins = ins + 1
    end
    println(data_dir_ins)
end
save(string(data_dir,"Input_Data.jld2"),Input_Data)

  0.000115 seconds (202 allocations: 652.484 KiB)
/Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Line_Design/N=2_N_x=12_N_u=5_S_train=200_N_nonzero=5/instance=1/
  0.000135 seconds (202 allocations: 652.484 KiB)
/Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Line_Design/N=2_N_x=12_N_u=5_S_train=200_N_nonzero=5/instance=2/
  0.000219 seconds (202 allocations: 652.484 KiB)
/Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Line_Design/N=2_N_x=12_N_u=5_S_train=200_N_nonzero=5/instance=3/
  0.000204 seconds (202 allocations: 652.484 KiB)
/Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Line_Design/N=2_N_x=12_N_u=5_S_train=200_N_nonzero=5/instance=4/
  0.000164 seconds (202 allocations: 652.484 KiB)
/Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Line_Design/N=2_N_x=12_N_u=5_S_train=200_N_nonzero=5/instance=5/
  0.000118 seconds (202 allocations: 652.484 KiB)
/Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Line_Design/N=2_N_x=12_N_u=5_S_tr

In [6]:
# ins = 1
# while ins <= instances
#     data_dir_ins = string(data_dir, "instance=$ins/")
#     # ****** Data ******
#     theta_true = Input_Data["theta_true_ins=$ins"]
#     r_params = Input_Data["r_params_ins=$ins"]
#     X_train = Input_Data["X_train_ins=$ins"]
#     Y_train = Input_Data["Y_train_ins=$ins"]
#     Z_train = Input_Data["Z_train_ins=$ins"]
#     X_test = Input_Data["X_test_ins=$ins"]
#     Y_test = Input_Data["Y_test_ins=$ins"]
#     Z_test = Input_Data["Z_test_ins=$ins"]
#     z_input = Z_test[1,:];
#     nu0_true,nu_true = compute_w(theta_true.alpha0,theta_true.alpha,theta_true.beta,theta_true.A,z_input)
#     w_true = [nu0_true;nu_true]

#     is_extreme_estimation = false
#     Estimate_Dict = Dict()
#     for lambda in lambda_list
#         alpha0_hat, alpha_hat, beta_hat, A_hat, opt_result = estimate_parameters(X_train,Y_train,Z_train,lambda, N_x, N_u, initial_theta=randn((N_x+1)*(N_u+1)) * 0.1);
#         nu0_hat,nu_hat = compute_w(alpha0_hat,alpha_hat,beta_hat,A_hat,z_input)
#         w_hat = [nu0_hat;nu_hat]
#         println("ins=$(ins)_w_hat=",round.(w_hat,digits=4))
#     end
#     ins = ins + 1
#     println(data_dir_ins)
# end

#### True

In [7]:
function get_input_data_ins(Input_Data,ins)
    theta_true = Input_Data["theta_true_ins=$ins"]
    r_params = Input_Data["r_params_ins=$ins"]
    X_train = Input_Data["X_train_ins=$ins"]
    Y_train = Input_Data["Y_train_ins=$ins"]
    Z_train = Input_Data["Z_train_ins=$ins"]
    X_test = Input_Data["X_test_ins=$ins"]
    Y_test = Input_Data["Y_test_ins=$ins"]
    Z_test = Input_Data["Z_test_ins=$ins"]
    r0 = r_params.r0;
    r = r_params.r;
    alp0_true = theta_true.alpha0;
    alp_true = theta_true.alpha;
    beta_true = theta_true.beta;
    A_true = theta_true.A;
    return r0,r,alp0_true,alp_true,beta_true,A_true,Z_test
end

get_input_data_ins (generic function with 1 method)

In [8]:
for ins in 1:instances
    data_dir_ins = string(data_dir, "instance=$ins/")
    r0,r,alp0_true,alp_true,beta_true,A_true,Z_test = get_input_data_ins(Input_Data,ins)
    nu0_true = Input_Data["nu0_ins=$ins"]
    nu_true = Input_Data["nu_ins=$ins"]
    w_true = [nu0_true;nu_true]

    Estimate_Dict = Input_Data["Estimate_ins=$ins"]
    l_index = 1
    lambda = lambda_list[l_index]

    nu0 = Estimate_Dict["nu0_lambda=$lambda"]
    nu = Estimate_Dict["nu_lambda=$lambda"]
    w_hat = [nu0;nu]
    println("diff=",round.(w_true .- w_hat,digits=2))
    println("norm = ",norm(w_hat .- w_true,2))

end

diff=[0.87, 0.41, 0.14, 0.33, -0.47, -0.42, -0.35, -0.43, -0.05, -0.42, -1.25, -0.34, 0.31]
norm = 1.9297389742327764
diff=[-2.34, -0.06, 0.77, -0.0, 0.38, 1.5, 2.21, 0.36, -1.32, -0.17, 1.43, -0.64, -0.81]
norm = 4.28135644496007
diff=[1.23, -0.51, 0.39, -0.7, 0.14, 0.86, 0.51, 0.74, -0.71, 0.26, -1.1, -0.46, 0.12]
norm = 2.4417355162394387
diff=[0.51, 0.09, 0.89, 0.6, -0.98, 0.23, -0.55, 0.52, 2.15, 0.54, -1.92, -0.48, -0.95]
norm = 3.5662311485367204
diff=[0.94, 0.94, -1.37, 0.77, -0.86, -0.44, -1.67, 1.05, 0.38, 0.03, -0.43, -0.96, 0.71]
norm = 3.28881087283613
diff=[0.49, -0.08, -0.42, 0.2, -0.19, 0.12, -0.63, -0.23, -0.76, 0.25, -1.32, -0.36, 0.59]
norm = 1.9556891708736057
diff=[-0.5, -0.74, 0.0, 0.35, 0.32, 0.44, 0.03, -0.64, -0.51, 1.82, 0.59, 0.33, -0.26]
norm = 2.394728191531908
diff=[2.12, -0.59, -1.29, -1.9, -0.29, -0.27, -0.32, 0.95, -0.12, 1.68, -1.45, 0.94, -0.06]
norm = 4.1387313527443
diff=[-0.87, 0.07, 0.53, 1.2, 0.0, 1.55, -0.23, -1.42, -1.26, 2.11, 0.28, 0.21, 1.72

In [9]:
Result_True_Dict = Dict();
for ins in 1:instances
    data_dir_ins = string(data_dir, "instance=$ins/")
    r0,r,alp0_true,alp_true,beta_true,A_true,Z_test = get_input_data_ins(Input_Data,ins)
    nu0 = Input_Data["nu0_ins=$ins"]
    nu = Input_Data["nu_ins=$ins"]

    obj_True_list = zeros(S_test);
    x_True_list = zeros(S_test,N,N_x);
    time_True_list = zeros(S_test);
    profit_True_list = zeros(S_test);
    for i in 1:S_test
        z_input = Z_test[i,:];
        obj_True_list[i], x_True_list[i,:,:], time_True_list[i] = ETO_PLD(N,N_x,nu0, nu, r0, r,c_l,d_r,rev_gap,num_c);
        profit_True_list[i] = calculate_profit(alp0_true, alp_true, beta_true, A_true, r0, r, x_True_list[i,:,:], z_input)
        # if i % 20 == 1
        #     println("True: i=$i, obj=$(round(obj_True_list[i], digits=4)), profit=$(round(profit_True_list[i], digits=4)), time=$(round(time_True_list[i],digits=4))")
        # end                         
    end
    println("************ data_dir_ins: ", data_dir_ins,"*************")
    Result_True_Dict["obj_ins=$ins"] = obj_True_list
    Result_True_Dict["sol_ins=$ins"] = x_True_list
    Result_True_Dict["time_ins=$ins"] = time_True_list
    Result_True_Dict["profit_ins=$ins"] = profit_True_list
end
save(string(data_dir,"Result_True.jld2"),Result_True_Dict) 

************ data_dir_ins: /Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Line_Design/N=2_N_x=12_N_u=5_S_train=200_N_nonzero=5/instance=1/*************
************ data_dir_ins: /Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Line_Design/N=2_N_x=12_N_u=5_S_train=200_N_nonzero=5/instance=2/*************
************ data_dir_ins: /Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Line_Design/N=2_N_x=12_N_u=5_S_train=200_N_nonzero=5/instance=3/*************
************ data_dir_ins: /Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Line_Design/N=2_N_x=12_N_u=5_S_train=200_N_nonzero=5/instance=4/*************
************ data_dir_ins: /Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Line_Design/N=2_N_x=12_N_u=5_S_train=200_N_nonzero=5/instance=5/*************
************ data_dir_ins: /Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Line_Design/N=2_N_x=12_N_u=5_S_train=200_N_nonzero=5/instance=6/*************
************ data_dir_ins: /

#### Estimate-then-optimize

In [10]:
Result_ETO_Dict = Dict();
for ins in 1:instances
    data_dir_ins = string(data_dir, "instance=$ins/")
    r0,r,alp0_true,alp_true,beta_true,A_true,Z_test = get_input_data_ins(Input_Data,ins)
    Estimate_Dict = Input_Data["Estimate_ins=$ins"]
    lambda_len = length(lambda_list)
    for l_index in 1:lambda_len
        lambda = lambda_list[l_index]

        nu0 = Estimate_Dict["nu0_lambda=$lambda"]
        nu = Estimate_Dict["nu_lambda=$lambda"]

        obj_ETO_list = zeros(S_test);
        x_ETO_list = zeros(S_test,N,N_x);
        time_ETO_list = zeros(S_test);
        profit_ETO_list = zeros(S_test);
        for i in 1:S_test
            z_input = Z_test[i,:];
            obj_ETO_list[i], x_ETO_list[i,:,:], time_ETO_list[i] = ETO_PLD(N,N_x,nu0, nu, r0, r,c_l,d_r,rev_gap,num_c);
            profit_ETO_list[i] = calculate_profit(alp0_true, alp_true, beta_true, A_true, r0, r, x_ETO_list[i,:,:], z_input)
            # if i % 20 == 1
            #     println("ETO: lambda=$lambda,i=$i, obj=$(round(obj_ETO_list[i], digits=4)), profit=$(round(profit_ETO_list[i], digits=4)),time=$(round(time_ETO_list[i],digits=4))")
            # end
        end
        Result_ETO_Dict["obj_ins=$(ins)_lambda=$lambda"] = obj_ETO_list
        Result_ETO_Dict["sol_ins=$(ins)_lambda=$lambda"] = x_ETO_list
        Result_ETO_Dict["time_ins=$(ins)_lambda=$lambda"] = time_ETO_list
        Result_ETO_Dict["profit_ins=$(ins)_lambda=$lambda"] = profit_ETO_list
    end
    println("************ data_dir_ins: ", data_dir_ins,"*************")
end
save(string(data_dir,"Result_ETO.jld2"),Result_ETO_Dict) 

************ data_dir_ins: /Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Line_Design/N=2_N_x=12_N_u=5_S_train=200_N_nonzero=5/instance=1/*************
************ data_dir_ins: /Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Line_Design/N=2_N_x=12_N_u=5_S_train=200_N_nonzero=5/instance=2/*************
************ data_dir_ins: /Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Line_Design/N=2_N_x=12_N_u=5_S_train=200_N_nonzero=5/instance=3/*************
************ data_dir_ins: /Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Line_Design/N=2_N_x=12_N_u=5_S_train=200_N_nonzero=5/instance=4/*************
************ data_dir_ins: /Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Line_Design/N=2_N_x=12_N_u=5_S_train=200_N_nonzero=5/instance=5/*************
************ data_dir_ins: /Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Line_Design/N=2_N_x=12_N_u=5_S_train=200_N_nonzero=5/instance=6/*************
************ data_dir_ins: /

#### Robust

In [11]:
function calculate_phi_bounds(N_x,r0,r,c_l,d_r_this,is_lb)
    model = Model(Mosek.Optimizer)
    set_attribute(model, "QUIET", true)
    # 定义变量
    @variable(model, X[1:N_x], Bin)        # 二进制变量 x_{jk}    
    @constraint(model, X' * c_l >= d_r_this)
    if is_lb
        @objective(model, Min, r0 + r' * X)
    else
        @objective(model, Max, r0 + r' * X)
    end
    optimize!(model)
    status = JuMP.termination_status(model)
    obj_val = objective_value(model)
    X_val = round.(value.(X))
    return obj_val,X_val
end

calculate_phi_bounds (generic function with 1 method)

In [12]:
# dual_norm = 2
# Result_RO_Dict = Dict();
# for ins in 1:instances
#     data_dir_ins = string(data_dir, "instance=$ins/")
#     r0,r,alp0_true,alp_true,beta_true,A_true,Z_test = get_input_data_ins(Input_Data,ins)
#     Estimate_Dict = Input_Data["Estimate_ins=$ins"]
#     lambda_len = length(lambda_list)
#     gamma_len = length(gamma_list)
#     for l_index in 1:lambda_len
#         lambda = lambda_list[l_index]
#         nu0 = Estimate_Dict["nu0_lambda=$lambda"]
#         nu = Estimate_Dict["nu_lambda=$lambda"]

#         for g_index in 1:gamma_len
#             gamma = gamma_list[g_index]
#             obj_RO_list = zeros(S_test);
#             x_RO_list = zeros(S_test,N,N_x);
#             time_RO_list = zeros(S_test);
#             profit_RO_list = zeros(S_test);
#             for i in 1:S_test
#                 z_input = Z_test[i,:];                
#                 r_lb,x_lb = calculate_phi_bounds(N_x,r0,r,c_l,d_r[1],true)
#                 r_ub,x_ub = calculate_phi_bounds(N_x,r0,r,c_l,d_r[1],false);
#                 phi_lb_cal = r_lb - r_ub
#                 psi_lb = -5 * ones(N)
#                 psi_ub = 0 * ones(N)
#                 phi_lb = -5 * ones(N)
#                 phi_ub = 0 * ones(N)

#                 obj_RO_list[i], x_RO_list[i,:,:], time_RO_list[i] = RO_PLD(N,N_x,nu0,nu,r0,r,c_l,d_r,rev_gap,psi_lb,psi_ub,phi_lb,phi_ub,gamma,dual_norm,num_c)
#                 profit_RO_list[i] = calculate_profit(alp0_true, alp_true, beta_true, A_true, r0, r, x_RO_list[i,:,:], z_input)
#                 # if i % 20 == 1
#                 #     println("RO: lambda=$lambda,gamma=$gamma, i=$i, obj=$(round(obj_RO_list[i], digits=4)), profit=$(round(profit_RO_list[i], digits=4)),x=$(round.(x_RO_list[i,:], digits=4)), time=$(round(time_RO_list[i],digits=4))")
#                 # end
#             end
#             Result_RO_Dict["obj_ins=$(ins)_lambda=$(lambda)_gamma=$gamma"] = obj_RO_list
#             Result_RO_Dict["sol_ins=$(ins)_lambda=$(lambda)_gamma=$gamma"] = x_RO_list
#             Result_RO_Dict["time_ins=$(ins)_lambda=$(lambda)_gamma=$gamma"] = time_RO_list
#             Result_RO_Dict["profit_ins=$(ins)_lambda=$(lambda)_gamma=$gamma"] = profit_RO_list
#         end
#     end
#     println("************ data_dir_ins: ", data_dir_ins,"*************")
# end
# save(string(data_dir,"Result_RO.jld2"),Result_RO_Dict) 

#### Results

In [13]:
Result_True_Dict = load(string(data_dir,"Result_True.jld2"));
Result_ETO_Dict = load(string(data_dir,"Result_ETO.jld2"));
# Result_RO_Dict = load(string(data_dir,"Result_RO.jld2"));

Dict{String, Any} with 400 entries:
  "profit_ins=45_lambda=0.001" => [2.0193]
  "time_ins=92_lambda=0.001"   => [0.109399]
  "profit_ins=68_lambda=0.001" => [2.31]
  "profit_ins=95_lambda=0.001" => [2.62965]
  "time_ins=81_lambda=0.001"   => [0.241497]
  "profit_ins=27_lambda=0.001" => [2.28413]
  "profit_ins=82_lambda=0.001" => [2.12609]
  "profit_ins=88_lambda=0.001" => [3.0483]
  "time_ins=38_lambda=0.001"   => [0.120697]
  "profit_ins=55_lambda=0.001" => [2.75944]
  "obj_ins=15_lambda=0.001"    => [2.53899]
  "profit_ins=61_lambda=0.001" => [1.50245]
  "profit_ins=62_lambda=0.001" => [2.83164]
  "profit_ins=65_lambda=0.001" => [2.47654]
  "time_ins=79_lambda=0.001"   => [0.163033]
  "obj_ins=45_lambda=0.001"    => [2.15676]
  "time_ins=13_lambda=0.001"   => [0.169384]
  "sol_ins=10_lambda=0.001"    => [1.0 -0.0;;; 0.0 1.0;;; 0.0 0.0;;; … ;;; 0.0 …
  "profit_ins=77_lambda=0.001" => [2.64378]
  ⋮                            => ⋮

In [14]:
# for k in keys(Result_RO_Dict)
#     if occursin("profit_ins=99", k)
#         println(k)
#     end
# end

In [15]:
# obj_True_avg = zeros(instances);
# obj_ETO_avg = zeros(instances, length(lambda_list));
# obj_RO_avg = zeros(instances, length(lambda_list), length(gamma_list));
# for ins in 1:instances
#     obj_True_avg[ins] = mean(Result_True_Dict["obj_ins=$ins"])
#     for l_index in 1:length(lambda_list)
#         lambda = lambda_list[l_index]
#         obj_ETO_avg[ins,l_index] = mean(Result_ETO_Dict["obj_ins=$(ins)_lambda=$lambda"])
#         for g_index in 1:length(gamma_list)
#             gamma = gamma_list[g_index]
#             obj_RO_avg[ins,l_index,g_index] = mean(Result_RO_Dict["obj_ins=$(ins)_lambda=$(lambda)_gamma=$gamma"])
#             # println("ins=$ins, lambda=$lambda, gamma=$gamma, profit_True_avg=$(round(profit_True_avg,digits=4)), profit_ETO_avg=$(round(profit_ETO_avg,digits=4)), profit_RO_avg=$(round(profit_RO_avg,digits=4))")
#         end
#     end
# end

In [16]:
# println("obj_ETO:",round.(obj_ETO_avg[:,1],digits=2))
# println("obj_RO:",round.(obj_RO_avg[:,1,1],digits=2))
# println("obj_ETO avg:",mean(obj_ETO_avg[:,1]))
# println("obj_RO avg:",mean(obj_RO_avg[:,1,1]))

In [17]:
profit_True_avg = zeros(instances);
profit_ETO_avg = zeros(instances, length(lambda_list));
profit_RO_avg = zeros(instances, length(lambda_list), length(gamma_list));
for ins in 1:instances
    profit_True_avg[ins] = mean(Result_True_Dict["profit_ins=$ins"])
    for l_index in 1:length(lambda_list)
        lambda = lambda_list[l_index]
        profit_ETO_avg[ins,l_index] = mean(Result_ETO_Dict["profit_ins=$(ins)_lambda=$lambda"])
        # for g_index in 1:length(gamma_list)
        #     gamma = gamma_list[g_index]
        #     profit_RO_avg[ins,l_index,g_index] = mean(Result_RO_Dict["profit_ins=$(ins)_lambda=$(lambda)_gamma=$gamma"])
        #     # println("ins=$ins, lambda=$lambda, gamma=$gamma, profit_True_avg=$(round(profit_True_avg,digits=4)), profit_ETO_avg=$(round(profit_ETO_avg,digits=4)), profit_RO_avg=$(round(profit_RO_avg,digits=4))")
        # end
    end
end

In [18]:
println("profit_ETO/profit_True = ",mean(profit_ETO_avg[:,1])/mean(profit_True_avg))
# println("profit_ETO:",round.(profit_ETO_avg[:,1],digits=2))
# println("profit_RO:",round.(profit_RO_avg[:,1,1],digits=2))

profit_ETO/profit_True = 0.9716969450143111


##### Line plot 

In [19]:
# l_index = 1
# RO_ETO_Ratio_Mean = zeros(length(gamma_list))
# RO_ETO_Ratio_Std = zeros(length(gamma_list))
# for g_index in 1:length(gamma_list)
#     gamma = gamma_list[g_index]
#     RO_ETO_Ratio_Mean[g_index] = mean(profit_RO_avg[:,l_index,g_index])/mean(profit_ETO_avg[:,l_index])
#     RO_ETO_Ratio_Std[g_index] = std(profit_RO_avg[:,l_index,g_index])/std(profit_ETO_avg[:,l_index])
#     println("gamma=$gamma,Mean =",RO_ETO_Ratio_Mean[g_index],", Std =",RO_ETO_Ratio_Std[g_index])
# end

In [20]:
# end_index = length(gamma_list)
# plot(gamma_list[1:end_index], RO_ETO_Ratio_Mean[1:end_index], marker=:o, xlabel=L"\gamma", label="Average Profit", xticks=(gamma_list[1:end_index], string.(gamma_list[1:end_index])))
# plot!(gamma_list[1:end_index], RO_ETO_Ratio_Std[1:end_index], marker=:o, xlabel=L"\gamma", ylabel="RPD/ETOPD", label="Std of Profit")
# hline!([1.0], linestyle=:dash, color=:red, label="")
# # savefig(string(data_dir, "RPD_vs_ETOPD.pdf"))

##### Boxplot

In [21]:
# Profit_ETO_All_Ins = profit_ETO_avg[:,1];
# Profit_RO_All_Ins = Dict();
# for g_index in 1:length(gamma_list)
#     gamma=gamma_list[g_index]
#     Profit_RO_All_Ins["gamma=$(gamma)"] = profit_RO_avg[:,1,g_index]
# end

In [22]:
# gamma_chosen = gamma_list[1:5]
# data = [Profit_ETO_All_Ins, [Profit_RO_All_Ins["gamma=$(gamma)"] for gamma in gamma_chosen]...]
# labels = ["ETO"; ["RO($gamma)" for gamma in gamma_chosen]]
# # --- 1) 展开成长表 ---
# df = DataFrame(
#     value = vcat(data...),
#     group = repeat(labels, inner = length(data[1]))
# )

# mycolors = palette(:tab10)[1:length(labels)]   # tab10 最多10个颜色

# # --- 3) 绘制箱线图 ---
# @df df boxplot(:group, :value;
#     group = :group,
#     palette = mycolors,
#     legend = false,
#     ylabel = "Revenue",
#     # title = "Revenue Distribution"
# )

# # --- 4) 计算均值 ---
# means = combine(groupby(df, :group), :value => mean => :mean_val)

# for i in 1:length(labels)
#     scatter!([labels[i]], [means.mean_val[i]];
#         color = :red,
#         marker = (:star, 10),
#         label = "")
# end

# # # --- 6) 绘制均值连线（这里必须把类别转为数值 1:N） ---
# plot!(labels, means.mean_val;
#     seriestype = :line,
#     color = :red,
#     lw = 2,
#     ls = :dash,
#     label = "Mean trend"
# )